In [12]:
# Check Eviatar result
#Import libraries
import pandas as pd
import numpy as np
import scipy
from scipy import integrate
from scipy.interpolate import CubicSpline
import glob
import re
import math
import os
import plotly.express as px
import plotly.graph_objects as go
#cross sections for O2 in cm^2
#O2 DisEx comes from Kanik et al.JGR, 108(E11),5126, doi:10.1029/2000JE001423,2003
#e- on O2 --> 1356
#Uses Table3 to 600 eV then follows shape function of 1304 above to end of eenergy range of file
asDEO21356_header = ["EO21356", "CS1356Mb"]
dfCSdeO21356 = pd.read_csv('DisExeO21356.csv',names=asDEO21356_header,skiprows=1)
dfCSdeO21356 = dfCSdeO21356.apply(pd.to_numeric)
dfCSdeO21356["CS1356Mb"] = dfCSdeO21356["CS1356Mb"]*1.0e-18
#Read in atmospheric densities as a afunction of altitude in meters
asAtm_header = ["alt", "H2On", "O2n", "H2n", "H2Om", "O2m", "H2m"]
dfAtm = pd.read_csv('Ganymede_09092021_Audrey_input2.csv',names=asAtm_header,skiprows=3)
dfAtm = dfAtm.apply(pd.to_numeric)

#Maxwellian flux function
def eflxblw13(e,Neval,Teval):
    rkbev = 8.6171e-5
    rkberg = 1.3805e-16
    elecmass = 9.1091e-28
    pi = 3.14159
    elecmeanvel = math.sqrt((8.0*rkberg*Teval)/(elecmass*pi))
    x = e/(rkbev*Teval)
    eflux = (Neval*elecmeanvel/4.0)*x*np.exp(-x)
    return eflux

# estimate measured electron density profiles with shape of Juno RSS and normalized to 331 cc value of Eviatar
#Function to determine the sample ionospheric density in cm-3 to use in this Eviatar comparison
def electrondensity(z):
    if z<1.5e7:
        edenval = 3.31e2*np.exp(-z/3.3e7)
    else:
        if z >= 3.5e7:
            edenval = 1.0e3*np.exp(-z/1.23e8)
        else:
            edenval = 1.7e3*np.exp(-z/3.8e7)
    return edenval
#Initialize altitude grid and O2 density
altgrd = np.array(dfAtm['alt'])
altgrd = np.array(altgrd*1.0e2)
no2den = np.array(dfAtm['O2n'])
no2den = np.array(no2den/1.0e+6)
#Read in Dissociative ionization cross sections
eno21356 = np.array(dfCSdeO21356["EO21356"],dtype=float)
cso21356 = np.array(dfCSdeO21356["CS1356Mb"],dtype=float)
nepts = len(eno21356)
csefo21356 = np.zeros_like(cso21356)
csefo2evtr1356 = np.zeros_like(cso21356)

for iz in range(len(altgrd)):

#Electron Flux interpolation scheme
    e=0.0
    Maxeflx = np.zeros(2000,dtype=float)
    earray = np.zeros(2000,dtype=float)
    edensity = electrondensity(altgrd[iz])
    for i in range(2000):
        earray[i] = (i+1)*1.0
        Maxeflx[i]=eflxblw13(earray[i],edensity,1.16e6)
    cs = CubicSpline(earray,Maxeflx)
#Maxwellian distribution plot
#figmax = px.scatter(x=Maxeflx,y=earray,log_x=1,title='electron flux (cm^-2 s^-1)',labels={"y":  "Energy(eV)", "x": "Flux(cm^-2 s^-1)"})
#figmax.add_scatter(x=Maxeflx,y=earray, mode='markers',name='Eviatar eflux')
#figmax.update_xaxes(exponentformat="E",title='Maxwellian distribution')
#figmax.show()
    for j in range(nepts):
        e = eno21356[j]
        efevtr = cs(e)
        csefo2evtr1356[j] = efevtr*cso21356[j]
#figmaxcs = px.scatter(x=csefo2evtr1356,y=eno21356,log_x=1,title='electron flux (cm^-2 s^-1)',labels={"y":  "Energy(eV)", "x": "Flux x Coross section(s^-1)"})
#figmaxcs.add_scatter(x=csefo2evtr1356,y=earray, mode='markers',name='Eviatar eflux times cross section for emission')
#figmaxcs.update_xaxes(exponentformat="E",title='Maxwellian distribution x Cross section')
#figmaxcs.show()
    pfo21356evtr = integrate.simpson(csefo2evtr1356,eno21356,even='avg')
    nevtradpo21356[iz] = no2den[iz]*pfo21356evtr

#Print and plot height integrated values and altitude profiles    


nevtro21356 = (integrate.simpson(nevtradpo21356,altgrd,even='avg'))/1.0e+6
print('noon eviatar o21356 in Rayleighs', "{:.2e}".format(nevtro21356))


figatmn = px.scatter(x=no2den, y=altgrd/1.0e5, log_x=1, title='Atmospheric densities (cm^-3)',labels={"y":  "Altitude(km)", "x": "Density(cm^-3)"})
figatmn.add_scatter(x=no2den, y=altgrd/1.0e5, mode='markers',name='Day O2')
figatmn.update_xaxes(exponentformat="E")
figatmn.show()

# Determine atmospheric column densities on dayside
colno2den = integrate.simpson(no2den,altgrd,even='avg')
print("NOON Column (cm^-2)",  "O2", "{:.2e}".format(colno2den))


figprod = px.scatter(x=nevtradpo21356,y=altgrd/1.0e5,log_x=1,title='photon production(cm^-3 s^-1)',labels={"y":  "Altitude (km)", "x": "Production(cm^-3 s^-1)"})
figprod.add_scatter(x=nevtradpo21356,y=altgrd/1.0e5, mode='markers',name='Eviatar photon production')
figprod.update_xaxes(exponentformat="E",title='Photon production')
figprod.show()


                  
                      
                      


noon eviatar o21356 in Rayleighs 1.13e+05


NOON Column (cm^-2) O2 4.52e+15
